In [16]:
Dataset = "FoCus"                      # "Blended Skill Talk", "IT-ConvAI2" , "FoCus" 

_COT = ""                                   #  "",    "-COT"

SCORING_METHOD = "train"                      # "train", "avg",  "length_prior", "benchmark"

LLM_name = f"gpt-3.5-turbo-{SCORING_METHOD}"        #Qwen2-7B-Instruct, gpt-3.5-turbo, gpt-4o-mini

In [17]:
LLM_name

'gpt-3.5-turbo-train'

In [18]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (1500, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"{\n ""response"": ""The HMS Plymouth is currentl...",0.916832
1,"{\n ""response"": ""As a fellow lighthouse enthu...",1.061527
2,"{\n ""response"": ""Acadiana is surrounded by ma...",0.891344
3,"{\n ""response"": ""The Vail Ski Resort in Color...",1.569819
4,"{\n ""response"": ""Mesa Verde National Park in ...",0.968816
...,...,...
1495,"{\n ""response"": ""The height of the Rocky Moun...",0.750636
1496,"{\n ""response"": ""Yes, the Rice Hotel is still...",0.647559
1497,"{\n ""response"": ""The most important discovery...",0.759824
1498,"{\n ""response"": ""Cardiff Castle hosts various...",35.994463


In [19]:
print(ds.iloc[167]["gen_response"])

{
  "response": "The opening ceremony of Shoreham Harbour Lifeboat Station was a significant event that marked the beginning of its operations and dedication to saving lives at sea."
}


In [20]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    if not isinstance(text, str):
        return None


    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     7
response_time    0
dtype: int64
(1500, 2)


,gen_response,response_time
0,The HMS Plymouth is currently a museum ship in...,0.916832
1,As a fellow lighthouse enthusiast who loves th...,1.061527
2,"Acadiana is surrounded by marshes, bayous, and...",0.891344
3,The Vail Ski Resort in Colorado offers ski les...,1.569819
4,Mesa Verde National Park in Colorado is known ...,0.968816
...,...,...
1495,"The height of the Rocky Mountains varies, with...",0.750636
1496,"Yes, the Rice Hotel is still open as a hotel, ...",0.647559
1497,The most important discovery in the Vulci area...,0.759824
1498,Cardiff Castle hosts various events throughout...,35.994463


In [21]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)